In [1]:
!pip install -qqq datasets==3.2.0 transformers==4.47.1 trl==0.14.0 peft==0.14.0 accelerate==1.2.1 bitsandbytes --progress-bar off
!pip install -qqq torch flash-attn --no-build-isolation --progress-bar off
!pip install -qqq sentence-transformers scikit-learn --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done


In [30]:
import torch
print(torch.__version__)

2.6.0+cu124


In [19]:
# If you run with GPU CUDA
!pip uninstall flash-attn
!pip install flash-attn --extra-index-url https://download.pytorch.org/whl/cu124

Found existing installation: flash_attn 2.8.0.post2
Uninstalling flash_attn-2.8.0.post2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/flash_attn-2.8.0.post2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/flash_attn/*
    /usr/local/lib/python3.11/dist-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so
    /usr/local/lib/python3.11/dist-packages/hopper/*
Proceed (Y/n)? y
  Successfully uninstalled flash_attn-2.8.0.post2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
  Using cached flash_attn-2.8.0.post2-cp311-cp311-linux_x86_64.whl


In [23]:
import os

In [27]:
os.environ["FLASH_ATTENTION_FORCE_DISABLED"] = "1"

In [28]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer
from datasets import concatenate_datasets
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# prompt: connect gg drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
root_path = '/content/drive/MyDrive/RL/gpt2'

In [5]:
ds = load_dataset("yonatanko/Relationship_Advice")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/401k [00:00<?, ?B/s]

dev.csv:   0%|          | 0.00/69.8k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/277k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/210 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [6]:
ds = ds.rename_column("post", "prompt")

def get_answer(example):
  more_helpful_comment = example["more_helpful_comment"].lower().replace(" ","_")
  if example["more_helpful_comment"] == "comment_1":
    example["answer"] = example["comment_1"]
  else:
    example["answer"] = example["comment_2"]
  return example

ds = ds.map(get_answer)
ds = ds.remove_columns(["comment_1", "comment_2", "comment_1_label", "comment_2_label", "more_helpful_comment", "batch", "example_id"])

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [7]:
ds['all'] = concatenate_datasets([ds['train'], ds['test'], ds['validation']])

In [8]:
ds['all']

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 400
})

In [29]:
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id)

RuntimeError: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
/usr/local/lib/python3.11/dist-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

In [10]:
if tokenizer.pad_token is not None and tokenizer.pad_token != tokenizer.eos_token:
    model.resize_token_embeddings(len(tokenizer))

In [11]:
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

trainable params: 2,359,296 || all params: 126,799,104 || trainable%: 1.8607
None


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [12]:
# Reward function
model_st = SentenceTransformer('all-MiniLM-L6-v2')

def reward_tfidf_similarity(completions, answer):
    vectorizer = TfidfVectorizer().fit([answer] + completions)
    vectors = vectorizer.transform([answer] + completions)
    sims = cosine_similarity(vectors[1:], vectors[0])
    return [float(1 - s[0]) for s in sims]

def reward_semantic_similarity(completions, answer, model=model_st, **kwargs):
    answer_embedding = model.encode(answer, convert_to_tensor=True)
    completion_embeddings = model.encode(completions, convert_to_tensor=True)
    similarity_scores = util.cos_sim(completion_embeddings, answer_embedding).squeeze(1)
    return [float(1 - score) for score in similarity_scores]

def reward_len(completions, answer, **kwargs):
    return [float(abs(len(completion) - len(answer))) for completion in completions]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
os.environ["WANDB_DISABLED"] = "true"

In [14]:
training_args = GRPOConfig(
    output_dir=os.path.join(root_path,"GRPO"),
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    max_prompt_length=512,
    max_completion_length=96,
    num_generations=8,
    optim="adamw_8bit",
    num_train_epochs=1,
    bf16=True,
    report_to=None,
    remove_unused_columns=False,
    logging_steps=1,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[reward_len,reward_tfidf_similarity,reward_semantic_similarity],
    args=training_args,
    train_dataset=ds["all"],
    processing_class=tokenizer
)

# Train model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3618: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  ctx_manager = torch.cpu.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
